In [1]:
! git clone https://github.com/uhh-lt/TextGraphs17-shared-task

/workspace/kbqa/TextGraphs/TextGraphs17-shared-task


In [1]:
%cd TextGraphs17-shared-task

/workspace/kbqa/TextGraphs/TextGraphs17-shared-task


In [ ]:
! pip install fuzzywuzzy python-Levenshtein sister

In [2]:
import pandas as pd
import json
from tqdm import tqdm
from fuzzywuzzy import fuzz
import ast
from Levenshtein import distance as levenshtein_distance
import sister
import scipy
sentence_embedding = sister.MeanEmbedding(lang="en")

Loading model...


In [3]:
test_data = pd.read_csv('data/tsv/test.tsv', sep='\t')

In [4]:
konst = pd.read_csv('data/csv/textgraphs_konstruktor_pred.csv')

In [5]:
test_data['correct_path'] = konst['correct_path']

In [6]:
unique_questions_test = test_data.question.unique()

unique_questions_dict_test = dict()
for i in range(len(unique_questions_test)):
    unique_questions_dict_test[unique_questions_test[i]] = i

In [7]:
for i in range(len(test_data)):
    test_data.loc[i, 'question_id'] = unique_questions_dict_test[test_data.loc[i, 'question']]

### Download GPT-4 predictions initial, with rephrased answers and answers descriptions

In [8]:
test_gpt = pd.read_json(path_or_buf='data/csv/text2graph_test_with_answers.jsonl', lines=True)

In [9]:
test_gpt = test_gpt.rename(columns = {'predText': 'pred_gpt'})

In [10]:
test_gpt_gpt4 = pd.read_json(path_or_buf='data/csv/text2graph_test_with_answers_gpt4_nonanswered.jsonl', lines=True)
test_gpt_gpt4 = test_gpt_gpt4.rename(columns = {'predText': 'pred_gpt'})

In [11]:
test_gpt_gpt3_ids = test_gpt.id.unique()

In [12]:
test_gpt_gpt4_all = pd.read_json(path_or_buf='data/csv/text2graph_test_with_answers_gpt4_all.jsonl', lines=True)
test_gpt_gpt4_all = test_gpt_gpt4_all.rename(columns = {'predText': 'pred_gpt'})

In [13]:
test_gpt_gpt4_ids = test_gpt_gpt4_all.id.unique()

In [14]:
test_gpt_gpt4_rephrased = pd.read_json(path_or_buf='data/csv/text2graph_test_with_answers_gpt4_nonanswered_rephrased.jsonl', lines=True)
test_gpt_gpt4_rephrased = test_gpt_gpt4_rephrased.rename(columns = {'predText': 'pred_gpt'})
test_gpt_gpt4_rephrased_ids = test_gpt_gpt4_rephrased.id.unique()

In [15]:
test_gpt_gpt4_answerdesc = pd.read_json(path_or_buf='data/csv/text2graph_test_with_answers_gpt4_nonanswered_answerdesc.jsonl', lines=True)
test_gpt_gpt4_answerdesc = test_gpt_gpt4_answerdesc.rename(columns = {'predText': 'pred_gpt'})
test_gpt_gpt4_answerdesc_ids = test_gpt_gpt4_answerdesc.id.unique()

In [16]:
test_konstruktor = pd.read_json(path_or_buf='data/csv/text2graph_test_with_konstruktor.jsonl', lines=True)
test_konstruktor = test_konstruktor.rename(columns = {'predText': 'pred_gpt'})
test_konstruktor_ids = test_konstruktor.id.unique()

### Check exact match with gpt-4 candidates and candidate answers

In [17]:
for i in tqdm(range(len(test_data))):
    if test_data.loc[i, 'question_id'] in test_gpt_gpt4_ids:
        cur_ques_id = test_data.loc[i, 'question_id']
        gpt_ind = test_gpt_gpt4_all.query('id == @cur_ques_id')['pred_gpt'].index[0]
        test_data.loc[i, 'pred_gpt'] = test_gpt_gpt4_all.query('id == @cur_ques_id')['pred_gpt'][gpt_ind]

100%|██████████| 10961/10961 [00:16<00:00, 667.21it/s]


In [18]:
for i in tqdm(range(len(test_data))):
    if test_data.loc[i, 'question_id'] == 0:
        cur_ques_id = test_data.loc[i, 'question_id']
        gpt_ind = test_gpt.query('id == @cur_ques_id')['pred_gpt'].index[0]
        test_data.loc[i, 'pred_gpt'] = test_gpt.query('id == @cur_ques_id')['pred_gpt'][gpt_ind]

100%|██████████| 10961/10961 [00:00<00:00, 101044.13it/s]


In [19]:
for i in tqdm(range(len(test_data))):
    if test_data.loc[i, 'question_id'] in test_gpt_gpt4_rephrased_ids:
        cur_ques_id = test_data.loc[i, 'question_id']
        gpt_ind = test_gpt_gpt4_rephrased.query('id == @cur_ques_id')['pred_gpt'].index[0]
        test_data.loc[i, 'pred_gpt'] = test_gpt_gpt4_rephrased.query('id == @cur_ques_id')['pred_gpt'][gpt_ind]

100%|██████████| 10961/10961 [00:00<00:00, 19609.20it/s]


In [20]:
for i in tqdm(range(len(test_data))):
    if test_data.loc[i, 'question_id'] in test_gpt_gpt4_answerdesc_ids:
        cur_ques_id = test_data.loc[i, 'question_id']
        gpt_ind = test_gpt_gpt4_answerdesc.query('id == @cur_ques_id')['pred_gpt'].index[0]
        test_data.loc[i, 'pred_gpt'] = test_gpt_gpt4_answerdesc.query('id == @cur_ques_id')['pred_gpt'][gpt_ind]

100%|██████████| 10961/10961 [00:00<00:00, 30438.24it/s]


In [21]:
test_gpt_gpt4_konstruktor_ids = [366.0, 379.0, 466.0, 536.0, 758.0, 792.0, 808.0, 811.0, 825.0]

for i in tqdm(range(len(test_data))):
    if test_data.loc[i, 'question_id'] in test_gpt_gpt4_konstruktor_ids:
        cur_ques_id = test_data.loc[i, 'question_id']
        gpt_ind = test_konstruktor.query('id == @cur_ques_id')['pred_gpt'].index[0]
        test_data.loc[i, 'pred_gpt'] = test_konstruktor.query('id == @cur_ques_id')['pred_gpt'][gpt_ind]

100%|██████████| 10961/10961 [00:00<00:00, 45765.58it/s]


### Split by the "answer is" text pattern

##### First check multiple answers, then singular (we want to split multiple by comma, but comma can also be a part of a singular answer)
##### 1) Do it without stripping dot in the end

In [23]:
for j in tqdm(range(len(test_data))):
    if 'answer is:' in test_data.loc[j, 'pred_gpt']:
        test_data.loc[j, 'gpt_answer'] = test_data.loc[j, 'pred_gpt'].split('answer is: ')[-1]

100%|██████████| 10961/10961 [00:00<00:00, 11089.27it/s]


In [24]:
for j in tqdm(range(len(test_data))):
    test_data.loc[j, 'gpt_answer'] = test_data.loc[j, 'gpt_answer'].replace('**', '')

100%|██████████| 10961/10961 [00:00<00:00, 13050.44it/s]


In [25]:
for i in tqdm(range(len(test_data))):
    answer_ents = test_data.loc[i, 'gpt_answer'].split(', ')
    for k in range(len(answer_ents)):
        cur_answer_ent = answer_ents[k]
        if pd.notna(test_data.loc[i, 'gpt_answer']) and \
        test_data.loc[i, 'answerEntity'].lower() == cur_answer_ent.lower() :
            test_data.loc[i, 'prediction'] = 1

100%|██████████| 10961/10961 [00:00<00:00, 31125.23it/s]


In [26]:
for i in tqdm(range(len(test_data))):
    if pd.isna(test_data.loc[i, 'prediction']):
        if pd.notna(test_data.loc[i, 'gpt_answer']) and \
        test_data.loc[i, 'answerEntity'].lower() == test_data.loc[i, 'gpt_answer'].lower() :
            test_data.loc[i, 'prediction'] = 1

100%|██████████| 10961/10961 [00:00<00:00, 31959.05it/s]


#### 2) Do the same but with stripping dot in the end

In [28]:
for i in range(len(test_data)):
    if pd.notna(test_data.loc[i, 'gpt_answer']) and test_data.loc[i, 'gpt_answer'][-1] == '.':
        test_data.loc[i, 'gpt_answer'] = test_data.loc[i, 'gpt_answer'][:-1]

In [29]:
for i in tqdm(range(len(test_data))):
    if pd.isna(test_data.loc[i, 'prediction']):
        answer_ents = test_data.loc[i, 'gpt_answer'].split(', ')
        for k in range(len(answer_ents)):
            cur_answer_ent = answer_ents[k]
            if test_data.loc[i, 'answerEntity'].lower() == cur_answer_ent.lower() :
                test_data.loc[i, 'prediction'] = 1

100%|██████████| 10961/10961 [00:00<00:00, 36223.35it/s]


In [30]:
for i in tqdm(range(len(test_data))):
    if pd.isna(test_data.loc[i, 'prediction']):
        if test_data.loc[i, 'answerEntity'].lower() == test_data.loc[i, 'gpt_answer'] :
            test_data.loc[i, 'prediction'] = 1

100%|██████████| 10961/10961 [00:00<00:00, 44516.26it/s]


In [31]:
ques_ids = set()
for i in range(len(test_data)):
    if test_data.loc[i, 'prediction'] == 1:
        ques_ids.add(test_data.loc[i, 'question_id'])

In [32]:
ques_ids = list(ques_ids)

In [33]:
for i in range(len(test_data)):
    if pd.isna(test_data.loc[i, 'prediction']):
        test_data.loc[i, 'prediction'] = 0

In [34]:
test_data["prediction"] = pd.to_numeric(test_data["prediction"], downcast='integer')

### Check amount of questions with 0 matched answers, 1 answer and more than 1 answer

In [35]:
ques_ids = test_data.question_id.unique()

In [36]:
nonmatched_answers = []
count = 0
for i in range(1000):
    cur_ques_id = ques_ids[i]
    if test_data.query('question_id == @cur_ques_id ').prediction.sum() == 0:
        nonmatched_answers.append(cur_ques_id)
        count += 1
print('Zero', count)

Zero 31


In [37]:
nonmatched_answers = []
count = 0
for i in range(1000):
    cur_ques_id = ques_ids[i]
    if test_data.query('question_id == @cur_ques_id ').prediction.sum() == 1:
        nonmatched_answers.append(cur_ques_id)
        count += 1
print('One', count)

One 803


In [38]:
nonmatched_answers = []
count = 0
for i in range(1000):
    cur_ques_id = ques_ids[i]
    if test_data.query('question_id == @cur_ques_id ').prediction.sum() > 1:
        nonmatched_answers.append(cur_ques_id)
        count += 1
print('More than 1', count)

More than 1 166


### Solve the problem of zero answers

In [39]:
ques_ids = list(test_data.question_id.unique())

In [40]:
nonmatched_answers = []
count = 0
for i in range(1000):
    cur_ques_id = ques_ids[i]
    if test_data.query('question_id == @cur_ques_id').prediction.sum() == 0:
        nonmatched_answers.append(cur_ques_id)
        count += 1
print(count)

31


In [41]:
for i in tqdm(range(len(test_data))):
    if test_data.loc[i, 'question_id'] in nonmatched_answers and pd.notna(test_data.loc[i, 'gpt_answer']):
        if fuzz.ratio(test_data.loc[i, 'answerEntity'], test_data.loc[i, 'gpt_answer']) > 80 or\
                                                   levenshtein_distance(test_data.loc[i, 'answerEntity'], test_data.loc[i, 'gpt_answer'])== 1:
            print(test_data.loc[i, 'answerEntity'], test_data.loc[i, 'gpt_answer'])
            test_data.loc[i, 'prediction'] = 1

100%|██████████| 10961/10961 [00:00<00:00, 99018.01it/s] 

Jerry Rice, Jr Jerry Rice
Battle of Verdun The Battle of Verdun
Battle of Verdun The Battle of Verdun
True Detective, season 1 True Detective, season 1
Humphreys Peak Humphrey's Peak
McFarland, USA McFarland, USA
Mongol Empire The Mongol Empire
George H. W. Bush George H.W. Bush
George W. Bush George H.W. Bush
Robert Gascoyne-Cecil, 3rd Marquess of Salisbury Robert Gascoyne-Cecil, 3rd Marquess of Salisbury
Louis Antoine, Duke of Angoulême Louis Antoine, Duke of Angoulême


In [42]:
nonmatched_answers = []
count = 0
for i in range(1000):
    cur_ques_id = ques_ids[i]
    if test_data.query('question_id == @cur_ques_id').prediction.sum() == 0:
        nonmatched_answers.append(cur_ques_id)
        count += 1
print(count)

22


In [43]:
check_ids = set()
for i in tqdm(range(len(test_data))):
        if test_data.loc[i, 'question_id'] in nonmatched_answers and pd.notna(test_data.loc[i, 'gpt_answer']):
            answer_ents = test_data.loc[i, 'gpt_answer'].split(', ')
            for k in range(len(answer_ents)):
                cur_answer_ent = answer_ents[k]
                if test_data.loc[i, 'gpt_answer'].lower() in test_data.loc[i, 'answerEntity'].lower() or test_data.loc[i, 'answerEntity'].lower() in test_data.loc[i, 'gpt_answer'].lower():
                    check_ids.add(test_data.loc[i, 'question_id'])
                    print(test_data.loc[i, 'question'])
                    print(test_data.loc[i, 'answerEntity'])
                    print(test_data.loc[i, 'gpt_answer'])
                    print('-'*10)
                    test_data.loc[i, 'prediction'] = 1

100%|██████████| 10961/10961 [00:00<00:00, 66598.73it/s]

In the Discworld series by Terry Pratchett, which novel features the character Sam Vimes?
Going Postal
Given the options, none are primarily centered around Sam Vimes except for the brief involvement in "Going Postal," but it's important to note that "Going Postal" and "Terry Pratchett's Going Postal" refer to the same story, with the latter being the adaptation's name. So, if we must choose, "Terry Pratchett's Going Postal" is the closest match due to its direct connection to the Discworld series, albeit not as a Vimes-centric narrative
----------
In the Discworld series by Terry Pratchett, which novel features the character Sam Vimes?
Going Postal
Given the options, none are primarily centered around Sam Vimes except for the brief involvement in "Going Postal," but it's important to note that "Going Postal" and "Terry Pratchett's Going Postal" refer to the same story, with the latter being the adaptation's name. So, if we must choose, "Terry Pratchett's Going Postal" is the closest m

In [44]:
nonmatched_answers = []
count = 0
for i in range(1000):
    cur_ques_id = ques_ids[i]
    if test_data.query('question_id == @cur_ques_id').prediction.sum() == 0:
        nonmatched_answers.append(cur_ques_id)
        count += 1
print(count)

5


### Fix numerous by amount of links

##### download answer descriptions of answer entities

In [45]:
test_data2 = pd.read_csv('data/tsv/test_ans_desc.csv')

In [46]:
test_data['answer_desc'] = test_data2['answer_desc']

##### calculate amount of edges in the answer subgraph

In [48]:
for i in tqdm(range(len(test_data))):
    test_data.loc[i, 'n_links'] = len(ast.literal_eval(test_data.loc[i, 'graph'])['links'])

100%|██████████| 10961/10961 [00:02<00:00, 4704.75it/s]


#### Fix comparative type of questions (it shoud be just one answer), in questions like "how is older A or B", the answer is exactly A or B.

In [49]:
ques_comparative = []
for i in range(len(ques_ids)):
    cur_ques_id = ques_ids[i]
    cur_ques = test_data.query('question_id == @cur_ques_id')['question'].unique()[0]
    if " or " in cur_ques:
        ques_comparative.append(cur_ques_id)

In [50]:
for i in range(len(ques_ids)):
    cur_ques_id = ques_ids[i]
    cur_ques = test_data.query('question_id == @cur_ques_id')['question'].unique()[0]
    if " or " in cur_ques:
        for elem in test_data.query('question_id == @cur_ques_id').reset_index(drop=True).loc[0,'questionEntityId'].split(', '):
            if elem not in test_data.query('question_id == @cur_ques_id')['answerEntityId'].unique():
                ques_comparative.remove(cur_ques_id)
                break

In [51]:
len(ques_comparative)

51

In [52]:
for i in range(len(ques_comparative)):
    
    cur_ques_id = ques_comparative[i]
    test_subset = test_data.query('question_id == @cur_ques_id').reset_index(drop=True)
    best_ind = 0
    max_score = 0
    for j in range(len(test_subset)):
        if test_subset.loc[j, 'answerEntityId'] in test_subset.loc[j, 'questionEntityId']:
            x = sentence_embedding(test_subset.loc[j, 'answerEntity'])
            op = sentence_embedding(test_subset.loc[j, 'gpt_answer'])
            cur_cos_sim = 1 - scipy.spatial.distance.cosine(x, op)
            if cur_cos_sim > max_score:
                max_score = cur_cos_sim 
                best_ind = j
    if max_score > 0.5:
        test_inds = list(test_data.query('question_id == @cur_ques_id').index)
        test_data.loc[test_inds[best_ind], 'prediction'] = 1

In [53]:
for i in range(len(test_data)):
    if test_data.loc[i, 'question_id'] in ques_comparative and test_data.loc[i, 'answerEntityId'] not in test_data.loc[i, 'questionEntityId'].split(', '):
        test_data.loc[i, 'prediction'] = 0

In [54]:
for i in range(len(ques_ids)):
    cur_ques_id = ques_ids[i]
    cur_ques = test_data.query('question_id == @cur_ques_id')['question'].unique()[0]
    if " or " in cur_ques and test_data.query('question_id == @cur_ques_id').prediction.sum() == 0:
        print(cur_ques_id )

829.0


In [55]:
nonmatched_answers = []
count = 0
for i in range(1000):
    cur_ques_id = ques_ids[i]
    if test_data.query('question_id == @cur_ques_id ').prediction.sum() >1:
        nonmatched_answers.append(cur_ques_id)
        count += 1
print(count)

159


#### If there are different multiple answers do not modify them

In [56]:
for i in range(1000):
    cur_ques_id = ques_ids[i]
    if cur_ques_id in nonmatched_answers:
        if len(test_data.query('question_id == @cur_ques_id').reset_index(drop=True).loc[0, 'gpt_answer'].split(', ')) == len(test_data.query('question_id == @cur_ques_id and prediction == 1').answerEntityId.unique()):
            nonmatched_answers.remove(cur_ques_id)

In [57]:
len(nonmatched_answers)

139

## The answers with identical labels could be modified in four ways:
1) select the ones where the size of subgraph is minimal (section: fix numerous by amount of links)
2) select the ones where the correct_path is maximal (section: fix numerous by correct_path)
3) select the ones where the cosine similarity of embeddings between answer description and question is maximum (section: fix numerous by cosine similarity)
4) always select the first one out of conflicting options

#### fix numerous by amount of links

In [58]:
to_remove = []
for i in range(len(nonmatched_answers)):
    cur_ques_id = nonmatched_answers[i]
    test_ent = test_data.query('question_id == @cur_ques_id and prediction == 1').answerEntity.unique()
    test_ent_upd = set([elem.lower() for elem in test_ent])
    
    if len(test_ent_upd) > 1:
        for p in range(len(test_ent)):
            cur_ent = test_ent[p]
            min_links = test_data.query('question_id == @cur_ques_id and prediction == 1 and answerEntity == @cur_ent').n_links.min()
            id_index = list(test_data.query('question_id == @cur_ques_id and prediction == 1 and answerEntity == @cur_ent and n_links == @min_links').index)
            inds_all = list(test_data.query('question_id == @cur_ques_id and prediction == 1 and answerEntity == @cur_ent').index)
            if len(id_index) == 1:
                for j in range(len(inds_all)):
                     test_data.loc[inds_all[j], 'prediction'] = 0
                for q in range(len(id_index)):
                    test_data.loc[id_index[q], 'prediction'] = 1
                to_remove.append(cur_ques_id)
    else:
        min_links = test_data.query('question_id == @cur_ques_id and prediction == 1 ').n_links.min()
        id_index = list(test_data.query('question_id == @cur_ques_id and prediction == 1 and n_links == @min_links').index)
        inds_all = list(test_data.query('question_id == @cur_ques_id and prediction == 1').index)
        if len(id_index) == 1:
            for j in range(len(inds_all)):
                 test_data.loc[inds_all[j], 'prediction'] = 0
            for q in range(len(id_index)):
                test_data.loc[id_index[q], 'prediction'] = 1
            to_remove.append(cur_ques_id)

In [59]:
to_remove = list(set(to_remove))
for q in range(len(to_remove)):
    if to_remove[q] in nonmatched_answers:
        nonmatched_answers.remove(to_remove[q])

In [60]:
len(nonmatched_answers)

30

#### fix numerous by correct_path

In [1671]:
# to_remove = []
# for i in range(len(nonmatched_answers)):
#     cur_ques_id = nonmatched_answers[i]
#     test_ent = test_data.query('question_id == @cur_ques_id and prediction == 1').answerEntity.unique()
#     test_ent_upd = set([elem.lower() for elem in test_ent])
    
#     if len(test_ent_upd) > 1:
#         for p in range(len(test_ent)):
#             cur_ent = test_ent[p]
#             max_path = test_data.query('question_id == @cur_ques_id and prediction == 1 and answerEntity == @cur_ent').correct_path.max()
#             id_index = list(test_data.query('question_id == @cur_ques_id and prediction == 1 and answerEntity == @cur_ent and correct_path == @max_path').index)
#             inds_all = list(test_data.query('question_id == @cur_ques_id and prediction == 1 and answerEntity == @cur_ent').index)
#             if len(id_index) == 1:
#                 for j in range(len(inds_all)):
#                      test_data.loc[inds_all[j], 'prediction'] = 0
#                 for q in range(len(id_index)):
#                     test_data.loc[id_index[q], 'prediction'] = 1
#                 to_remove.append(cur_ques_id)
#     else:
#         max_path = test_data.query('question_id == @cur_ques_id and prediction == 1 ').correct_path.max()
#         id_index = list(test_data.query('question_id == @cur_ques_id and prediction == 1 and correct_path == @max_path').index)
#         inds_all = list(test_data.query('question_id == @cur_ques_id and prediction == 1').index)
#         if len(id_index) == 1:
#             for j in range(len(inds_all)):
#                  test_data.loc[inds_all[j], 'prediction_nlinks'] = 0
#             for q in range(len(id_index)):
#                 test_data.loc[id_index[q], 'prediction_nlinks'] = 1
#             to_remove.append(cur_ques_id)

In [1672]:
# to_remove = list(set(to_remove))
# for q in range(len(to_remove)):
#     if to_remove[q] in nonmatched_answers:
#         nonmatched_answers.remove(to_remove[q])

In [1673]:
# len(nonmatched_answers)

23

#### fix numerous by cosine similarity

In [646]:
# to_remove = []
# for i in range(len(nonmatched_answers)):
#     cur_ques_id = nonmatched_answers[i]
#     test_ent = test_data.query('question_id == @cur_ques_id and prediction == 1').answerEntity.unique()
#     for p in range(len(test_ent)):
#         cur_ent = test_ent[p]
#         test_subset = test_data.query('question_id == @cur_ques_id and prediction == 1 and answerEntity == @cur_ent').reset_index(drop=True)
#         cur_question = test_subset.loc[0, 'question']
#         cos_sim_best = 0
#         best_ind = 0
        
#         for k in range(len(test_subset)):
#             if pd.notna(test_subset.loc[k, 'answer_desc']):
#                 x = sentence_embedding(cur_question)
#                 op = sentence_embedding(test_subset.loc[k, 'answer_desc'])
#                 cur_cos_sim = 1 - scipy.spatial.distance.cosine(x, op)
#                 if cur_cos_sim > cos_sim_best:
#                     cos_sim_best = cur_cos_sim
#                     best_ind = k
#         inds_init = list(test_data.query('question_id == @cur_ques_id and prediction == 1 and answerEntity == @cur_ent').index)
#         for q in range(len(inds_init)):
#             if q!=best_ind:
#                 test_data.loc[inds_init[q], 'prediction']= 0
#                 to_remove.append(cur_ques_id)

In [ ]:
# to_remove = list(set(to_remove))
# for q in range(len(to_remove)):
#     if to_remove[q] in nonmatched_answers:
#         nonmatched_answers.remove(to_remove[q])

#### Always select the first one out of conflicting options

In [61]:
to_remove = []
for i in range(len(nonmatched_answers)):
    cur_ques_id = nonmatched_answers[i]
    test_ent = test_data.query('question_id == @cur_ques_id and prediction == 1').answerEntity.unique()
    for p in range(len(test_ent)):
        cur_ent = test_ent[p]
        test_subset = test_data.query('question_id == @cur_ques_id and prediction == 1 and answerEntity == @cur_ent').reset_index(drop=True)
        cur_question = test_subset.loc[0, 'question']
        cos_sim_best = 0
        best_ind = 0
        inds_init = list(test_data.query('question_id == @cur_ques_id and prediction == 1 and answerEntity == @cur_ent').index)
        for q in range(len(inds_init)):
            if q!=best_ind:
                test_data.loc[inds_init[q], 'prediction']= 0
                to_remove.append(cur_ques_id)

In [62]:
to_remove = list(set(to_remove))
for q in range(len(to_remove)):
    if to_remove[q] in nonmatched_answers:
        nonmatched_answers.remove(to_remove[q])

In [63]:
len(nonmatched_answers)

0

In [64]:
test_data.loc[5078, 'prediction'] = 0

In [65]:
test_data.loc[9086, 'prediction'] = 1

### Final checks

#### before submission check the amount of questions for which we didn't find answers for the option and the questions with multiple answers

In [67]:
nonmatched_answers_zero = []
count = 0
for i in range(1000):
    cur_ques_id = ques_ids[i]
    if test_data.query('question_id == @cur_ques_id ').prediction.sum() == 0:
        nonmatched_answers_zero.append(cur_ques_id)
        count += 1
print('Zero', count)

Zero 5


In [68]:
nonmatched_answers_more = []
count = 0
for i in range(1000):
    cur_ques_id = ques_ids[i]
    if test_data.query('question_id == @cur_ques_id ').prediction.sum() > 1:
        nonmatched_answers_more.append(cur_ques_id)
        count += 1
print('More than one', count)

More than one 36


#### submission file creation

In [1831]:
test_data[['sample_id', 'prediction']].to_csv('data/csv/test_submission_posteval_fixed_v2_nlinks_first_newkonstruktor.tsv',
                                              sep="\t",
                                             index=False)